In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from src.data_paths import *

In [2]:
df = pd.read_pickle(MERGED)
cosmic_proteins = pd.read_csv(COSMIC_PROTEINS, sep='\t')

In [3]:
cosmic_proteins.head(1)

,Gene,Gene synonym,Ensembl,Gene description,Uniprot,Chromosome,Position,Protein class,Biological process,Molecular function,...,Cancer prognostics - Pancreatic Adenocarcinoma (TCGA),Cancer prognostics - Pancreatic Adenocarcinoma (validation),Cancer prognostics - Prostate Adenocarcinoma (TCGA),Cancer prognostics - Rectum Adenocarcinoma (TCGA),Cancer prognostics - Rectum Adenocarcinoma (validation),Cancer prognostics - Skin Cuteneous Melanoma (TCGA),Cancer prognostics - Stomach Adenocarcinoma (TCGA),Cancer prognostics - Testicular Germ Cell Tumor (TCGA),Cancer prognostics - Thyroid Carcinoma (TCGA),Cancer prognostics - Uterine Corpus Endometrial Carcinoma (TCGA)
0,ABI1,"ABI-1, E3B1, SSH3BP1",ENSG00000136754,Abl interactor 1,Q8IZP0,10,26746593-26861087,"Cancer-related genes, Disease related genes, P...",Host-virus interaction,NaN,...,unprognostic (6.17e-3),unprognostic (9.66e-2),unprognostic (8.60e-2),unprognostic (1.16e-1),unprognostic (3.28e-2),unprognostic (1.01e-1),unprognostic (3.66e-1),unprognostic (1.72e-1),unprognostic (1.03e-1),unprognostic (1.72e-1)


### Check if synonyms could be used

In [4]:
arr = cosmic_proteins['Gene synonym'].apply(lambda x: np.array([y.strip() for y in str(x).split(',')]))
all_elems = np.concatenate(arr)

# Check isin for each element in the array (careful of double lists)
df['target_name'].isin(all_elems).sum(), df['drugbank_protein_name'].isin(all_elems).sum()

(0, 0)

In [5]:
cosmic_proteins['Uniprot']

0      Q8IZP0
1      P00519
2      P42684
3      P25106
4      O95573
        ...  
567    Q9UBW7
568    Q9NQX6
569    Q8TF68
570    Q96K83
571    Q15696
Name: Uniprot, Length: 572, dtype: object

In [6]:
# Matching by drugbank_protein_name
nb_bind_in_cosmic_bn = df['drugbank_protein_name'].isin(cosmic_proteins['Gene']).sum()
nb_cosmic_in_bind_bn = cosmic_proteins['Gene'].isin(df['drugbank_protein_name']).sum()
nb_bind_in_cosmic_by_targetname_bn = df['target_name'].isin(cosmic_proteins['Gene']).sum()

# Matching by uniprot
nb_bind_in_cosmic_bu = df['swissprot_protein_id'].dropna().isin(cosmic_proteins['Uniprot']).sum()
nb_cosmic_in_bind_bu = cosmic_proteins['Uniprot'].isin(df['swissprot_protein_id'].dropna()).sum()

print(f"Number of bind proteins in cosmic by drugbank_protein_name: {nb_bind_in_cosmic_bn}")
print(f"Number of cosmic proteins in bind by drugbank_protein_name: {nb_cosmic_in_bind_bn}")
print()
print(f"Number of bind proteins in cosmic by target_name: {nb_bind_in_cosmic_by_targetname_bn}")
print()
print(f"Number of bind proteins in cosmic by uniprot: {nb_bind_in_cosmic_bu}")
print(f"Number of cosmic proteins in bind by uniprot: {nb_cosmic_in_bind_bu}")

Number of bind proteins in cosmic by drugbank_protein_name: 20199
Number of cosmic proteins in bind by drugbank_protein_name: 45

Number of bind proteins in cosmic by target_name: 0

Number of bind proteins in cosmic by uniprot: 32320
Number of cosmic proteins in bind by uniprot: 75


### Cancer related proteins

In [7]:
bind_cancer_proteins_by_name = df[df['drugbank_protein_name'].isin(cosmic_proteins['Gene'])]
bind_cancer_proteins_by_uniprot = df[df['swissprot_protein_id'].isin(cosmic_proteins['Uniprot'])]

bcpbn_set = set(bind_cancer_proteins_by_name['target_name'])
bcpbu_set = set(bind_cancer_proteins_by_uniprot['target_name'])

print(f"Number of cancer proteins in bind by drugbank_protein_name: {len(bcpbn_set)}")
print(f"Number of cancer proteins in bind by uniprot: {len(bcpbu_set)}")
print(f"Difference between the two sets:")
print(f"  - {len(bcpbn_set - bcpbu_set)} proteins in drugbank_protein_name but not in uniprot")
print(f"  - {len(bcpbu_set - bcpbn_set)} proteins in uniprot but not in drugbank_protein_name")

Number of cancer proteins in bind by drugbank_protein_name: 91
Number of cancer proteins in bind by uniprot: 142
Difference between the two sets:
  - 4 proteins in drugbank_protein_name but not in uniprot
  - 55 proteins in uniprot but not in drugbank_protein_name


In [8]:
cancer_proteins_namey_not_uniprot = df[df['target_name'].isin(bcpbn_set - bcpbu_set)]
cancer_proteins_namey_not_uniprot[['drugbank_protein_name', 'target_name', 'swissprot_protein_id']]

,drugbank_protein_name,target_name,swissprot_protein_id
64111,EGFR,"Phosphatidylinositol 4,5-bisphosphate 3-kinase...",Q504U8
64125,EGFR,Receptor protein-tyrosine kinase/Serine/threon...,Q504U8
64139,EGFR,DNA-dependent protein kinase catalytic subunit...,Q504U8
64153,EGFR,"Phosphatidylinositol 4,5-bisphosphate 3-kinase...",Q504U8


In [9]:
cosmic_proteins[cosmic_proteins['Gene'] == 'EGFR'][['Gene', 'Gene synonym', 'Uniprot']]

,Gene,Gene synonym,Uniprot
141,EGFR,"ERBB, ERBB1, ERRP",P00533


In [10]:
# Combine the two sets (even though Uniprot is not the same)
cancer_related_proteins = pd.concat([bind_cancer_proteins_by_name, bind_cancer_proteins_by_uniprot]).drop_duplicates()
cancer_related_proteins_df = df[df['target_name'].isin(cancer_related_proteins['target_name'])]

print(f"Number of pairs of cancer proteins with their ligand in bind: {len(cancer_related_proteins_df)}")
print(f"Number of unique cancer proteins: {len(cancer_related_proteins['target_name'].unique())}")
print(f"  - {len(bind_cancer_proteins_by_name['target_name'].unique())} by drugbank_protein_name")
print(f"  - {len(bind_cancer_proteins_by_uniprot['target_name'].unique())} by uniprot")

Number of pairs of cancer proteins with their ligand in bind: 67058
Number of unique cancer proteins: 146
  - 91 by drugbank_protein_name
  - 142 by uniprot


### Ligands related to these proteins

In [62]:
ligands_related_to_cancer_proteins = cancer_related_proteins_df.dropna(subset=['ligand_name'])

print(f"Number of pairs of ligands related to cancer proteins (keep only the ones for which we have a name): {len(ligands_related_to_cancer_proteins)}")
print(f"Number of unique ligands related to cancer proteins: {len(set(ligands_related_to_cancer_proteins['ligand_name']))}")
print(f"Number of cancer proteins that matched to a ligand: {len(set(ligands_related_to_cancer_proteins['target_name']))} out of {len(cancer_related_proteins['target_name'].unique())}")

Number of pairs of ligands related to cancer proteins (keep only the ones for which we have a name): 67058
Number of unique ligands related to cancer proteins: 39196
Number of cancer proteins that matched to a ligand: 146 out of 146


In [63]:
# Keep only the ligands that are in DrugBank 
drugs_related_to_cancer_proteins = ligands_related_to_cancer_proteins.dropna(subset='drugbank_drug_class_superclass')

print(f"Number of pairs of drugs related to cancer proteins: {len(drugs_related_to_cancer_proteins)}")
print(f"Number of unique drugs related to cancer proteins: {len(drugs_related_to_cancer_proteins['drugbank_drug_name'].unique())}")
print(f"Number of cancer proteins that matched to a drug: {len(set(drugs_related_to_cancer_proteins['target_name']))} out of {len(cancer_related_proteins['target_name'].unique())}")

Number of pairs of drugs related to cancer proteins: 3192
Number of unique drugs related to cancer proteins: 321
Number of cancer proteins that matched to a drug: 68 out of 146


### All proteins related to these ligands

In [75]:
all_prots_related_to_cancer_ligands = df[df['ligand_name'].isin(ligands_related_to_cancer_proteins['ligand_name'].unique())]
all_prots_related_to_cancer_drugs = df[df['drugbank_drug_name'].isin(drugs_related_to_cancer_proteins['drugbank_drug_name'].unique())]

print(f"Number of unique proteins related to cancer ligands: {len(all_prots_related_to_cancer_ligands['target_name'].unique())}")
print(f"Number of unique proteins related to cancer drugs: {len(all_prots_related_to_cancer_drugs['target_name'].unique())}")

Number of unique proteins related to cancer ligands: 2092
Number of unique proteins related to cancer drugs: 1592


### Differentiate between secondary effects and drugs

In [79]:
direct_prots_related_target_names = cancer_related_proteins['target_name'].unique()

direct_effect_prots = all_prots_related_to_cancer_drugs[all_prots_related_to_cancer_drugs['target_name'].isin(direct_prots_related_target_names)]
secondary_effect_prots = all_prots_related_to_cancer_drugs[~all_prots_related_to_cancer_drugs['target_name'].isin(direct_prots_related_target_names)]

ligands_direct_effect_prots = all_prots_related_to_cancer_ligands[all_prots_related_to_cancer_ligands['target_name'].isin(direct_prots_related_target_names)]
ligands_secondary_effect_prots = all_prots_related_to_cancer_ligands[~all_prots_related_to_cancer_ligands['target_name'].isin(direct_prots_related_target_names)]

print(f"Number of proteins directly targeted by drugs: {len(direct_effect_prots['target_name'].unique())}")
print(f"Number of proteins representing secondary effect of drugs: {len(secondary_effect_prots['target_name'].unique())}")
print()
print(f"Number of proteins directly targeted by ligands: {len(ligands_direct_effect_prots['target_name'].unique())}")
print(f"Number of proteins representing secondary effect of ligands: {len(ligands_secondary_effect_prots['target_name'].unique())}")

Number of proteins directly targeted by drugs: 68
Number of proteins representing secondary effect of drugs: 1524

Number of proteins directly targeted by ligands: 146
Number of proteins representing secondary effect of ligands: 1946


Note that cancer related proteins has 146 distinct prots, but as we keep only the ones that are linked to a ligand in bindingdb, we end up using only 66 (61 for drugs) out of those 146 prots. So in the end, the difference between number of proteins directly targeted by ligands and the number of proteins that are related to cancer is only because of this:
```	python
    ligands_related_to_cancer_proteins = cancer_related_proteins_df.dropna(subset=['drugbank_drug_name'])
```
In summary, 
- we have 146 cancer prots
- they link to 405 ligands (ligands related to cancer prots), which themselves only link to 66 of the cancer prots (61 for drugs)
- these 405 ligands link to all prots which around 1651 prots
- out of all these prots, as we've seen 66 are cancer related (61 for drugs), and 1504 are likely proteins on which the drug has secondary effects

The difference between the 146 cancer prots and the 66 cancer prots is really because only a subset of the cancer prots can be used for our analysis. And this subset, as explained above, is given by the dropna on drugbank_drug_name which is the name of the ligand.

In [86]:
direct_effect_prots['drugbank_drug_name'].isin(all_prots_related_to_cancer_drugs['drugbank_drug_name']).sum()

3192

In [ ]:

# Create a list of nodes
nodes = []
node_id_map = {}  # To map node names to unique ids
categories = [{'name': 'Possible Cancer Drug'}, {'name': 'Cancer Protein'}, {'name': 'Secondary Effect Protein'}]

# Process ligand nodes
ligand_nodes = all_prots_related_to_cancer_drugs['drugbank_drug_name'].dropna().unique()
for idx, ligand in enumerate(ligand_nodes):
    ligand = str(ligand)
    node_id_map[ligand] = idx
    node = {
        'id': str(idx),
        'name': ligand,
        'category': 0,  # Index of 'ligand' in categories
        'symbolSize': 10,  # Adjust as needed
        'value': 1  # You can set this to degree or other measure
    }
    nodes.append(node)

# Process protein nodes
cancer_protein_nodes = direct_effect_prots['target_name'].unique()
for idx, protein in enumerate(cancer_protein_nodes, start=len(node_id_map)):
    node_id_map[protein] = idx
    node = {
        'id': str(idx),
        'name': protein,
        'category': 1,  # Index of 'protein' in categories
        'symbolSize': 10,  # Adjust as needed
        'value': 1  # You can set this to degree or other measure
    }
    nodes.append(node)

secondary_protein_nodes = secondary_effect_prots['target_name'].unique()
for idx, protein in enumerate(secondary_protein_nodes, start=len(node_id_map)):
    node_id_map[protein] = idx
    node = {
        'id': str(idx),
        'name': protein,
        'category': 2,  # Index of 'protein' in categories
        'symbolSize': 10,  # Adjust as needed
        'value': 1  # You can set this to degree or other measure
    }
    nodes.append(node)


In [96]:
# Create a list of edges
edges = []
for _, row in direct_effect_prots.iterrows():
    source_id = node_id_map[row['drugbank_drug_name']]
    target_id = node_id_map[row['target_name']]
    edge = {
        'source': str(source_id),
        'target': str(target_id),
        'value': row['ic50']  # Use ic50 as edge value
    }
    edges.append(edge)

for _, row in secondary_effect_prots.iterrows():
    source_id = node_id_map[row['drugbank_drug_name']]
    target_id = node_id_map[row['target_name']]
    edge = {
        'source': str(source_id),
        'target': str(target_id),
        'value': row['ic50']  # Use ic50 as edge value
    }
    edges.append(edge)

# Create the graph data
graph_data = {
    'nodes': nodes,
    'links': edges,
    'categories': categories
}

In [ ]:
from pyecharts.charts import Graph
from pyecharts import options as opts


graph = Graph(init_opts=opts.InitOpts(width="100%", height="700px"))
graph.add(
    "",
    nodes,
    edges,
    categories=categories,
    layout="force",
    edge_length=[50, 200],
    repulsion=100,
    linestyle_opts=opts.LineStyleOpts(width=0.5, opacity=0.5),
)
graph.set_series_opts(
    label_opts=opts.LabelOpts(
        is_show=False,  # Hide labels by default
        position="right",
        formatter="{b}",  # Use node name as the label
        #show=False,  # Ensure labels are not rendered statically
    )
)

graph.set_global_opts(
    title_opts=opts.TitleOpts(title="Cancer Related Ligand-Protein Interaction Network"),
)

graph.render("graph.html")


'c:\\Users\\sebge\\source\\test2\\ada-2024-project-hyperbabbage\\graph.html'